In [1]:
import numpy as np
import scipy
import h5py
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.python.keras.optimizers import Adam,RMSprop 

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Sequential

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Sequential,model_from_json
from PIL import Image

C:\Users\ali\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
STARTDIR='C:\\data\\clothes\\'
adversarialexample_dir = os.path.join(STARTDIR,'adversarialexample\\')
model_dir = os.path.join(STARTDIR,'model_ourclothes.json')
weights_dir=os.path.join(STARTDIR,'weights_ourclothes.h5')
image_width, image_height = 150, 150
learning_rate = 0.3  ## How much to update the hacked image in each iteration
object_type_to_fake = 1  #Pant
input_shape = (image_width, image_height, 3) 
labels = ['hat', 'pant', 'shirt', 'shoes', 'skirt', 'socks']

In [3]:
json_file = open(model_dir, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(weights_dir)
print(loaded_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_37 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
activation_38 (Activation)   (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 36, 36, 64)        0         
__________

In [5]:
files = os.listdir(adversarialexample_dir)
    
for file in files:
        fullFileName= os.path.join(adversarialexample_dir, file)
        print(fullFileName)
        model_input_layer = loaded_model.layers[0].input  # First Layer
        model_output_layer = loaded_model.layers[-1].output # Last Layer        
        img = image.load_img(fullFileName, target_size=(image_width, image_height))
        original_image = image.img_to_array(img)

        # Scale the image so all pixel intensities are between [-1, 1] as the model expects
        original_image /= 255.

        # Add a 4th dimension for batch size (as Keras expects)
        original_image = np.expand_dims(original_image, axis=0)
        #Create a copy of the input image to hack on
        adversarialEg_image = np.copy(original_image)
        
        # Pre-calculate the maximum change we will allow to the image
        max_change_above = original_image + 0.09
        max_change_below = original_image - 0.09

        # Define the cost function.
        # Our 'cost' will be the likelihood out image is the target class according to the pre-trained model
        cost_function = model_output_layer[0, object_type_to_fake]

        # We'll ask Keras to calculate the gradient based on the input image and the currently predicted class
        # In this case, referring to "model_input_layer" will give us back image we are hacking.
        gradient_function = K.gradients(cost_function, model_input_layer)[0]

        # Create a Keras function that we can call to calculate the current cost and gradient
        grab_cost_and_gradients_from_model = K.function([model_input_layer, K.learning_phase()], [cost_function, gradient_function])

        cost = 0.0     
        
        # In a loop, keep adjusting the adversarialEg image slightly so that it tricks the model more and more
        # until it gets to at least 90% confidence
        while cost < 0.80:
            # Check how close the image is to our target class and grab the gradients we
            # can use to push it one more step in that direction.
            # Note: It's really important to pass in '0' for the Keras learning mode here!
            # Keras layers behave differently in prediction vs. train modes!
            cost, gradients = grab_cost_and_gradients_from_model([adversarialEg_image, 0])

            # Move the adversarialEg image one step further towards fooling the model
            adversarialEg_image += gradients * learning_rate

            # Ensure that the image doesn't ever change too much to either look funny or to become an invalid image
            adversarialEg_image = np.clip(adversarialEg_image, max_change_below, max_change_above)
            adversarialEg_image = np.clip(adversarialEg_image, -1.0, 1.0)

            print("Adversarial Example Image cost is {:.3}%".format(cost * 100))

        # De-scale the image's pixels from [-1, 1] back to the [0, 255] range
        img = adversarialEg_image[0]
        img *= 255.
        # Save the adversarialEg image!
        im = Image.fromarray(img.astype(np.uint8))
        file = "advEx_"+file
        adversarialEgFileName= os.path.join(adversarialexample_dir, file)
        print(adversarialEgFileName)
        im.save(adversarialEgFileName)

C:\data\clothes\adversarialexample\userhat_002.jpg
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.136%
Adversarial Example Image cost is 0.137%
Adversarial Example Image cost is 0.137%
Adversarial Example Image cost is 0.137%
Adversarial Example Image cost is 0.137%
Advers

Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.148%
Adversarial Example Image cost is 0.149%
Adversarial Example Image cost is 0.149%
Adversarial Example Image cost is 0.149%
Adversarial Example Image cost is 0.149%
Adversarial Example Image cost is 0.149%
Adversarial Example Image cost is 0.149%
Adversarial Example Image cost is 0.149%
Adversarial Example Image cost is 0.149%
Adversarial Exam

Adversarial Example Image cost is 0.161%
Adversarial Example Image cost is 0.161%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.162%
Adversarial Example Image cost is 0.163%
Adversarial Example Image cost is 0.163%
Adversarial Example Image cost is 0.163%
Adversarial Example Image cost is 0.163%
Adversarial Example Image cost is 0.163%
Adversarial Example Image cost is 0.163%
Adversarial Example Image cost is 0.163%
Adversarial Example Image cost is 0.163%
Adversarial Exam

Adversarial Example Image cost is 0.178%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.179%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image cost is 0.18%
Adversarial Example Image co

Adversarial Example Image cost is 0.199%
Adversarial Example Image cost is 0.199%
Adversarial Example Image cost is 0.199%
Adversarial Example Image cost is 0.199%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.2%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.201%
Adversarial Example Image cost is 0.202%
Adversarial Example Image cost is 

...

Adversarial Example Image cost is 0.6%
Adversarial Example Image cost is 0.601%
Adversarial Example Image cost is 0.602%
Adversarial Example Image cost is 0.603%
Adversarial Example Image cost is 0.604%
Adversarial Example Image cost is 0.605%
Adversarial Example Image cost is 0.606%
Adversarial Example Image cost is 0.607%
Adversarial Example Image cost is 0.608%
Adversarial Example Image cost is 0.609%
Adversarial Example Image cost is 0.61%
Adversarial Example Image cost is 0.611%
Adversarial Example Image cost is 0.612%
Adversarial Example Image cost is 0.612%
Adversarial Example Image cost is 0.613%
Adversarial Example Image cost is 0.614%
Adversarial Example Image cost is 0.615%
Adversarial Example Image cost is 0.616%
Adversarial Example Image cost is 0.617%
Adversarial Example Image cost is 0.618%
Adversarial Example Image cost is 0.619%
Adversarial Example Image cost is 0.62%
Adversarial Example Image cost is 0.621%
Adversarial Example Image cost is 0.622%
Adversarial Example 

Adversarial Example Image cost is 0.915%
Adversarial Example Image cost is 0.917%
Adversarial Example Image cost is 0.919%
Adversarial Example Image cost is 0.922%
Adversarial Example Image cost is 0.924%
Adversarial Example Image cost is 0.926%
Adversarial Example Image cost is 0.928%
Adversarial Example Image cost is 0.931%
Adversarial Example Image cost is 0.933%
Adversarial Example Image cost is 0.935%
Adversarial Example Image cost is 0.938%
Adversarial Example Image cost is 0.94%
Adversarial Example Image cost is 0.943%
Adversarial Example Image cost is 0.945%
Adversarial Example Image cost is 0.947%
Adversarial Example Image cost is 0.95%
Adversarial Example Image cost is 0.952%
Adversarial Example Image cost is 0.955%
Adversarial Example Image cost is 0.957%
Adversarial Example Image cost is 0.96%
Adversarial Example Image cost is 0.962%
Adversarial Example Image cost is 0.965%
Adversarial Example Image cost is 0.967%
Adversarial Example Image cost is 0.97%
Adversarial Example 

Adversarial Example Image cost is 1.92%
Adversarial Example Image cost is 1.93%
Adversarial Example Image cost is 1.94%
Adversarial Example Image cost is 1.95%
Adversarial Example Image cost is 1.96%
Adversarial Example Image cost is 1.97%
Adversarial Example Image cost is 1.98%
Adversarial Example Image cost is 1.99%
Adversarial Example Image cost is 2.0%
Adversarial Example Image cost is 2.01%
Adversarial Example Image cost is 2.02%
Adversarial Example Image cost is 2.03%
Adversarial Example Image cost is 2.04%
Adversarial Example Image cost is 2.05%
Adversarial Example Image cost is 2.06%
Adversarial Example Image cost is 2.08%
Adversarial Example Image cost is 2.09%
Adversarial Example Image cost is 2.1%
Adversarial Example Image cost is 2.11%
Adversarial Example Image cost is 2.12%
Adversarial Example Image cost is 2.13%
Adversarial Example Image cost is 2.15%
Adversarial Example Image cost is 2.16%
Adversarial Example Image cost is 2.17%
Adversarial Example Image cost is 2.19%
Ad

Adversarial Example Image cost is 15.8%
Adversarial Example Image cost is 16.0%
Adversarial Example Image cost is 16.3%
Adversarial Example Image cost is 16.5%
Adversarial Example Image cost is 16.8%
Adversarial Example Image cost is 17.1%
Adversarial Example Image cost is 17.4%
Adversarial Example Image cost is 17.7%
Adversarial Example Image cost is 18.0%
Adversarial Example Image cost is 18.3%
Adversarial Example Image cost is 18.6%
Adversarial Example Image cost is 18.9%
Adversarial Example Image cost is 19.2%
Adversarial Example Image cost is 19.5%
Adversarial Example Image cost is 19.8%
Adversarial Example Image cost is 20.1%
Adversarial Example Image cost is 20.4%
Adversarial Example Image cost is 20.8%
Adversarial Example Image cost is 21.1%
Adversarial Example Image cost is 21.5%
Adversarial Example Image cost is 21.8%
Adversarial Example Image cost is 22.1%
Adversarial Example Image cost is 22.5%
Adversarial Example Image cost is 22.9%
Adversarial Example Image cost is 23.2%


Adversarial Example Image cost is 71.5%
Adversarial Example Image cost is 71.6%
Adversarial Example Image cost is 71.8%
Adversarial Example Image cost is 71.9%
Adversarial Example Image cost is 72.0%
Adversarial Example Image cost is 72.2%
Adversarial Example Image cost is 72.3%
Adversarial Example Image cost is 72.4%
Adversarial Example Image cost is 72.6%
Adversarial Example Image cost is 72.7%
Adversarial Example Image cost is 72.8%
Adversarial Example Image cost is 73.0%
Adversarial Example Image cost is 73.1%
Adversarial Example Image cost is 73.2%
Adversarial Example Image cost is 73.3%
Adversarial Example Image cost is 73.4%
Adversarial Example Image cost is 73.5%
Adversarial Example Image cost is 73.7%
Adversarial Example Image cost is 73.8%
Adversarial Example Image cost is 73.9%
Adversarial Example Image cost is 74.0%
Adversarial Example Image cost is 74.1%
Adversarial Example Image cost is 74.2%
Adversarial Example Image cost is 74.3%
Adversarial Example Image cost is 74.5%
